In [1]:
%%bash 
#wget -N https://ikpublictutorial.blob.core.windows.net/deeplearningframeworks/tf-densenet121.tar.gz
#tar xzvf tf-densenet121.tar.gz

In [2]:
import os
import sys
import time
import multiprocessing
import numpy as np
import pandas as pd
import tensorflow as tf
from nets import densenet  # Download from https://github.com/pudae/tensorflow-densenet
from tensorflow.python.framework import dtypes
from tensorflow.python.framework.ops import convert_to_tensor
from tensorflow.contrib.data import Iterator
#from tensorflow.contrib.slim.python.slim.nets import resnet_v1
from PIL import Image
import random
from common.utils import download_data_chextxray, get_imgloc_labels, get_train_valid_test_split
from common.utils import compute_roc_auc
slim = tf.contrib.slim

In [3]:
tf.__version__

'1.4.0'

In [4]:
CPU_COUNT = multiprocessing.cpu_count()
print("CPUs: ", CPU_COUNT)
NUM_GPU = 2

CPUs:  12


In [5]:
# Globals
CLASSES = 14
WIDTH = 224
HEIGHT = 224
CHANNELS = 3
LR = 0.0001 * NUM_GPU # Effective learning-rate will decrease as BATCHSIZE rises
EPOCHS = 5
BATCHSIZE = 64 * NUM_GPU
IMAGENET_RGB_MEAN = np.array([123.68, 116.78, 103.94], dtype=np.float32)
IMAGENET_SCALE_FACTOR = 0.017
TOT_PATIENT_NUMBER = 30805  # From data

In [6]:
# Paths
CSV_DEST = "chestxray"
IMAGE_FOLDER = os.path.join(CSV_DEST, "images")
LABEL_FILE = os.path.join(CSV_DEST, "Data_Entry_2017.csv")
print(IMAGE_FOLDER, LABEL_FILE)
# Model checkpoint
CHKPOINT = 'tf-densenet121.ckpt'
#CHKPOINT = 'resnet_v1_50.ckpt'

chestxray/images chestxray/Data_Entry_2017.csv


In [7]:
%%time
# Download data
print("Please make sure to download")
print("https://docs.microsoft.com/en-us/azure/storage/common/storage-use-azcopy-linux#download-and-install-azcopy")
download_data_chextxray(CSV_DEST)

Please make sure to download
https://docs.microsoft.com/en-us/azure/storage/common/storage-use-azcopy-linux#download-and-install-azcopy
Data already exists
CPU times: user 758 ms, sys: 206 ms, total: 964 ms
Wall time: 965 ms


In [8]:
#####################################################################################################
## Data Loading

In [9]:
class XrayData():
    
    def __init__(self, img_dir, lbl_file, patient_ids, mode='inference', 
                 width=WIDTH, height=HEIGHT, batch_size=BATCHSIZE, 
                 imagenet_mean=IMAGENET_RGB_MEAN, imagenet_scaling = IMAGENET_SCALE_FACTOR):
        # Get data
        self.img_locs, self.labels = get_imgloc_labels(img_dir, lbl_file, patient_ids)
        self.data_size = len(self.labels)
        self.imagenet_mean = imagenet_mean
        self.imagenet_scaling = imagenet_scaling
        self.width = width
        self.height = height
        # Create dataset
        # Performance: https://www.tensorflow.org/versions/master/performance/datasets_performance
        # Following: https://stackoverflow.com/a/48096625/6772173
        data = tf.data.Dataset.from_tensor_slices((self.img_locs, self.labels))
        # Processing
        if mode == 'training':
            print("Training")
            data = data.shuffle(self.data_size).repeat().map(self._parse_function_train,
                            num_parallel_calls=CPU_COUNT).prefetch(10*batch_size).batch(batch_size)
        else:
            print("Inference")
            data = data.map(self._parse_function_inference,
                            num_parallel_calls=CPU_COUNT).prefetch(10*batch_size).batch(batch_size)
        
        self.data = data        
        print("Loaded {} labels and {} images".format(len(self.labels), len(self.img_locs)))
        
        
    def _parse_function_train(self, filename, label):
        img_rgb, label = self._preprocess_image_labels(filename, label)
        # Super high CPU usuage bottlenecking GPU
        # Random crop
        img_rgb = tf.image.resize_images(img_rgb, [self.height+40, self.width+40])
        img_rgb = tf.random_crop(img_rgb, [self.height, self.width, 3])
        # Random flip
        img_rgb = tf.image.random_flip_left_right(img_rgb)
        return img_rgb, label
        
        
    def _parse_function_inference(self, filename, label):
        img_rgb, label = self._preprocess_image_labels(filename, label)
        # Resize to final dimensions
        img_rgb = tf.image.resize_images(img_rgb, [self.height, self.width])
        return img_rgb, label 
       
    
    def _preprocess_image_labels(self, filename, label):
        # load and preprocess the image
        img_decoded = tf.to_float(tf.image.decode_png(tf.read_file(filename), channels=3))
        img_centered = tf.subtract(img_decoded, self.imagenet_mean)
        img_rgb = img_centered * self.imagenet_scaling
        return img_rgb, tf.cast(label, dtype=tf.float32)

In [10]:
train_set, valid_set, test_set = get_train_valid_test_split(TOT_PATIENT_NUMBER)

train:21563 valid:3080 test:6162


In [11]:
with tf.device('/cpu:0'):
    # Create dataset for iterator
    train_dataset = XrayData(img_dir=IMAGE_FOLDER, lbl_file=LABEL_FILE, patient_ids=train_set,  
                             mode='training')
    valid_dataset = XrayData(img_dir=IMAGE_FOLDER, lbl_file=LABEL_FILE, patient_ids=valid_set)
    test_dataset  = XrayData(img_dir=IMAGE_FOLDER, lbl_file=LABEL_FILE, patient_ids=test_set)

Training
Loaded 87306 labels and 87306 images
Inference
Loaded 7616 labels and 7616 images
Inference
Loaded 17198 labels and 17198 images


In [12]:
#####################################################################################################
## Helper Functions

In [13]:
def average_gradients(tower_grads):
    # https://github.com/BobLiu20/Classification_Nets/blob/master/tensorflow/common/average_gradients.py
    average_grads = []
    for grad_and_vars in zip(*tower_grads):
        grads = []
        for g, _ in grad_and_vars:
            expanded_g = tf.expand_dims(g, 0)
            grads.append(expanded_g)
        # Average over the 'tower' dimension.
        grad = tf.concat(axis=0, values=grads)
        grad = tf.reduce_mean(grad, 0)
        # Keep in mind that the Variables are redundant because they are shared
        # across towers. So .. we will just return the first tower's pointer to
        # the Variable.
        v = grad_and_vars[0][1]
        grad_and_var = (grad, v)
        average_grads.append(grad_and_var)
    return average_grads

In [14]:
def get_symbol(model_name, in_tensor, is_training, chkpoint, out_features):
    if model_name == 'resnet50':
        # Import symbol
        with slim.arg_scope(resnet_v1.resnet_arg_scope()):
            base_model, _ = resnet_v1.resnet_v1_50(in_tensor, 
                                                   num_classes=None, 
                                                   is_training=is_training)
        # Attach extra layers
        fc = tf.layers.dense(base_model, out_features, name='output')
        # Activation function will be included in loss
        sym = tf.reshape(fc, shape=[-1, out_features])
        # Load weights for layers
        variables_to_restore = slim.get_variables_to_restore(exclude=['output'])
    elif model_name == 'densenet121':
         # Import symbol
        with slim.arg_scope(densenet.densenet_arg_scope()):
            base_model, _ = densenet.densenet121(in_tensor,
                                                 num_classes=out_features,
                                                 is_training=is_training)
            # Need to reshape from (?, 1, 1, 14) to (?, 14)
            sym = tf.reshape(base_model, shape=[-1, out_features])
        # Collect variables to restore from checkpoint
        variables_to_restore = slim.get_variables_to_restore(exclude=['densenet121/logits', 'predictions'])
    else:
        raise ValueError("Unknown model-name")
    return sym, variables_to_restore

In [15]:
def model_fn(features, labels, mode, params):
    # Create symbol
    # is_training=True?
    # https://github.com/tensorflow/models/issues/3556
    sym, variables_to_restore = get_symbol(
        model_name=params["model_name"],
        in_tensor=features, 
        #is_training=(mode == tf.estimator.ModeKeys.TRAIN),
        is_training=True,
        chkpoint=params["checkpoint"],
        out_features=params["n_classes"])
    # Predictions
    predictions = tf.sigmoid(sym)   
    # ModeKeys.PREDICT
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, 
                                          predictions=predictions)
    # Optimizer & Loss
    optimizer = tf.train.AdamOptimizer(params['lr'], beta1=0.9, beta2=0.999)
    loss_fn = tf.losses.sigmoid_cross_entropy(labels, sym)
    loss = tf.reduce_mean(loss_fn)
    if mode == tf.estimator.ModeKeys.TRAIN:
        #https://stackoverflow.com/questions/47867748/transfer-learning-with-tf-estimator-estimator-framework
        tf.train.init_from_checkpoint(params['checkpoint'], 
                              {v.name.split(':')[0]: v for v in variables_to_restore})
    return tf.estimator.EstimatorSpec(
        mode=mode,
        loss=loss,
        train_op=optimizer.minimize(loss, tf.train.get_or_create_global_step()))

In [16]:
def model_fn_multigpu(features, labels, mode, params):
    # todo ...
    # https://github.com/BobLiu20/Classification_Nets/blob/master/tensorflow/training/train_estimator.py
    if mode == tf.estimator.ModeKeys.PREDICT:
        # Create symbol
        # Predict on one-GPU
        sym, _ = get_symbol(
            model_name=params["model_name"],
            in_tensor=features, 
            is_training=True,
            chkpoint=params["checkpoint"],
            out_features=params["n_classes"])
        
        # Predictions
        predictions = tf.sigmoid(sym)   
        # ModeKeys.PREDICT
        return tf.estimator.EstimatorSpec(mode=mode, 
                                          predictions=predictions)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        # Make sure splits sum to batch-size
        split_size = params["batchsize"] // len(params["gpus"])
        splits = [split_size, ] * (len(params["gpus"]) - 1)
        splits.append(params["batchsize"] - split_size * (
            len(params["gpus"]) - 1))
        # Split the features and labels
        features_split = tf.split(features, splits, axis=0)
        labels_split = tf.split(labels, splits, axis=0)
        tower_grads = []
        # Training operation
        global_step = tf.train.get_global_step()
        optimizer = tf.train.AdamOptimizer(LR, beta1=0.9, beta2=0.999)
        # Load model on multiple GPUs
        restore_list = []
        with tf.variable_scope(tf.get_variable_scope()):
            for i in range(len(params['gpus'])):
                with tf.device('/gpu:%d' % i):
                    with tf.name_scope('%s_%d' % ("classification", i)) as scope:
                        sym, variables_to_restore = get_symbol(
                            model_name=params["model_name"],
                            in_tensor=features_split[i], 
                            is_training=True,
                            chkpoint=params["checkpoint"],
                            out_features=params["n_classes"])
                        restore_list.append(variables_to_restore)
                        
                        tf.losses.sigmoid_cross_entropy(labels_split[i], sym)

                        update_ops = tf.get_collection(
                            tf.GraphKeys.UPDATE_OPS, scope)
                        updates_op = tf.group(*update_ops)

                        with tf.control_dependencies([updates_op]):
                            losses = tf.get_collection(tf.GraphKeys.LOSSES, scope)
                            print(losses)
                            total_loss = tf.add_n(losses, name='total_loss')
                        # reuse var
                        tf.get_variable_scope().reuse_variables()
                        # grad compute
                        grads = optimizer.compute_gradients(total_loss)
                        tower_grads.append(grads)

        # We must calculate the mean of each gradient
        grads = average_gradients(tower_grads)
        # Apply the gradients to adjust the shared variables.
        apply_gradient_op = optimizer.apply_gradients(grads, global_step=global_step)
        # Group all updates to into a single train op.
        train_op = tf.group(apply_gradient_op)
        # Initialise from checkpoint
        tf.train.init_from_checkpoint(params['checkpoint'], 
                              {v.name.split(':')[0]: v for v in restore_list[0]})
        
        return tf.estimator.EstimatorSpec(
            mode=mode,
            loss=total_loss,
            train_op=train_op)

In [17]:
def train_input_fn():
    return train_dataset.data.make_one_shot_iterator().get_next()
def valid_input_fn():
    return valid_dataset.data.make_one_shot_iterator().get_next()
def test_input_fn():
    return test_dataset.data.make_one_shot_iterator().get_next()

In [18]:
%%time
# Create Estimator
nn = tf.estimator.Estimator(model_fn=model_fn_multigpu,
                            params={"lr":LR, 
                                    "checkpoint":CHKPOINT,
                                    "n_classes":CLASSES,
                                    "batchsize":BATCHSIZE,
                                    "gpus":[0,1],
                                    "model_name":"densenet121"})

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_num_ps_replicas': 0, '_is_chief': True, '_master': '', '_num_worker_replicas': 1, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f760a105b38>, '_session_config': None, '_tf_random_seed': None, '_service': None, '_save_checkpoints_steps': None, '_model_dir': '/tmp/tmp6esvh3lq', '_log_step_count_steps': 100, '_task_type': 'worker', '_keep_checkpoint_every_n_hours': 10000, '_task_id': 0, '_save_summary_steps': 100, '_keep_checkpoint_max': 5, '_save_checkpoints_secs': 600}
CPU times: user 4.35 ms, sys: 63 µs, total: 4.41 ms
Wall time: 3.93 ms


In [19]:
%%time
# 1 GPU - Main training loop: 58mins 8s
# 2 GPU - Main training loop: 46mins 14s
# What's a good way of adding validation data here?
nn.train(train_input_fn, 
         max_steps=EPOCHS*(train_dataset.data_size//BATCHSIZE))

[<tf.Tensor 'classification_0/sigmoid_cross_entropy_loss/value:0' shape=() dtype=float32>]
[<tf.Tensor 'classification_1/sigmoid_cross_entropy_loss/value:0' shape=() dtype=float32>]
INFO:tensorflow:Initialize variable densenet121/dense_block3/conv_block23/x2/Conv/weights:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block3/conv_block23/x2/Conv/weights
INFO:tensorflow:Initialize variable densenet121/dense_block4/conv_block5/x1/BatchNorm/beta:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block4/conv_block5/x1/BatchNorm/beta
INFO:tensorflow:Initialize variable densenet121/dense_block2/conv_block1/x1/BatchNorm/gamma:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block2/conv_block1/x1/BatchNorm/gamma
INFO:tensorflow:Initialize variable densenet121/dense_block1/conv_block6/x2/BatchNorm/moving_mean:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block1/conv_block6/x2/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable densenet121/

INFO:tensorflow:Initialize variable densenet121/dense_block3/conv_block8/x1/BatchNorm/moving_mean:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block3/conv_block8/x1/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable densenet121/dense_block2/conv_block4/x2/BatchNorm/moving_variance:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block2/conv_block4/x2/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable densenet121/dense_block2/conv_block4/x1/BatchNorm/moving_variance:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block2/conv_block4/x1/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable densenet121/dense_block3/conv_block24/x1/BatchNorm/beta:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block3/conv_block24/x1/BatchNorm/beta
INFO:tensorflow:Initialize variable densenet121/dense_block1/conv_block5/x1/BatchNorm/gamma:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block1/conv_block5/x1/BatchNorm

INFO:tensorflow:Initialize variable densenet121/dense_block2/conv_block1/x2/Conv/weights:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block2/conv_block1/x2/Conv/weights
INFO:tensorflow:Initialize variable densenet121/dense_block3/conv_block7/x1/BatchNorm/moving_variance:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block3/conv_block7/x1/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable densenet121/dense_block4/conv_block7/x1/BatchNorm/beta:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block4/conv_block7/x1/BatchNorm/beta
INFO:tensorflow:Initialize variable densenet121/dense_block4/conv_block16/x2/BatchNorm/moving_variance:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block4/conv_block16/x2/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable densenet121/dense_block2/conv_block5/x2/Conv/weights:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block2/conv_block5/x2/Conv/weights
INFO:tensorflow:I

INFO:tensorflow:Initialize variable densenet121/dense_block2/conv_block12/x2/Conv/weights:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block2/conv_block12/x2/Conv/weights
INFO:tensorflow:Initialize variable densenet121/dense_block3/conv_block17/x1/BatchNorm/beta:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block3/conv_block17/x1/BatchNorm/beta
INFO:tensorflow:Initialize variable densenet121/dense_block2/conv_block3/x2/BatchNorm/gamma:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block2/conv_block3/x2/BatchNorm/gamma
INFO:tensorflow:Initialize variable densenet121/dense_block2/conv_block4/x2/BatchNorm/moving_mean:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block2/conv_block4/x2/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable densenet121/dense_block4/conv_block15/x2/BatchNorm/gamma:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block4/conv_block15/x2/BatchNorm/gamma
INFO:tensorflow:Initialize variable

INFO:tensorflow:Initialize variable densenet121/dense_block4/conv_block15/x2/BatchNorm/beta:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block4/conv_block15/x2/BatchNorm/beta
INFO:tensorflow:Initialize variable densenet121/dense_block3/conv_block12/x1/BatchNorm/moving_mean:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block3/conv_block12/x1/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable densenet121/dense_block3/conv_block21/x1/BatchNorm/beta:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block3/conv_block21/x1/BatchNorm/beta
INFO:tensorflow:Initialize variable densenet121/BatchNorm/gamma:0 from checkpoint tf-densenet121.ckpt with densenet121/BatchNorm/gamma
INFO:tensorflow:Initialize variable densenet121/dense_block4/conv_block16/x2/Conv/weights:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block4/conv_block16/x2/Conv/weights
INFO:tensorflow:Initialize variable densenet121/dense_block2/conv_block12/x1/Conv/weights:0

...

INFO:tensorflow:Initialize variable densenet121/dense_block4/conv_block2/x1/BatchNorm/moving_variance:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block4/conv_block2/x1/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable densenet121/dense_block3/conv_block18/x2/BatchNorm/gamma:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block3/conv_block18/x2/BatchNorm/gamma
INFO:tensorflow:Initialize variable densenet121/dense_block3/conv_block22/x2/BatchNorm/moving_mean:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block3/conv_block22/x2/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable densenet121/dense_block2/conv_block11/x1/BatchNorm/beta:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block2/conv_block11/x1/BatchNorm/beta
INFO:tensorflow:Initialize variable densenet121/dense_block3/conv_block12/x2/BatchNorm/beta:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block3/conv_block12/x2/BatchNorm/beta
INFO:tens

INFO:tensorflow:Initialize variable densenet121/dense_block3/conv_block15/x2/BatchNorm/moving_mean:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block3/conv_block15/x2/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable densenet121/dense_block3/conv_block19/x1/BatchNorm/moving_variance:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block3/conv_block19/x1/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable densenet121/dense_block1/conv_block6/x2/Conv/weights:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block1/conv_block6/x2/Conv/weights
INFO:tensorflow:Initialize variable densenet121/dense_block1/conv_block3/x1/BatchNorm/gamma:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block1/conv_block3/x1/BatchNorm/gamma
INFO:tensorflow:Initialize variable densenet121/final_block/BatchNorm/beta:0 from checkpoint tf-densenet121.ckpt with densenet121/final_block/BatchNorm/beta
INFO:tensorflow:Initialize variable densenet121/d

INFO:tensorflow:Initialize variable densenet121/dense_block3/conv_block11/x2/BatchNorm/gamma:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block3/conv_block11/x2/BatchNorm/gamma
INFO:tensorflow:Initialize variable densenet121/dense_block4/conv_block8/x1/BatchNorm/moving_variance:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block4/conv_block8/x1/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable densenet121/dense_block4/conv_block10/x2/BatchNorm/gamma:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block4/conv_block10/x2/BatchNorm/gamma
INFO:tensorflow:Initialize variable densenet121/dense_block2/conv_block8/x2/BatchNorm/moving_variance:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block2/conv_block8/x2/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable densenet121/dense_block1/conv_block1/x2/BatchNorm/beta:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block1/conv_block1/x2/BatchNorm/beta
INF

INFO:tensorflow:Initialize variable densenet121/dense_block2/conv_block4/x1/BatchNorm/gamma:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block2/conv_block4/x1/BatchNorm/gamma
INFO:tensorflow:Initialize variable densenet121/dense_block1/conv_block6/x1/BatchNorm/moving_variance:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block1/conv_block6/x1/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable densenet121/dense_block3/conv_block4/x1/BatchNorm/gamma:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block3/conv_block4/x1/BatchNorm/gamma
INFO:tensorflow:Initialize variable densenet121/dense_block1/conv_block6/x2/BatchNorm/moving_variance:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block1/conv_block6/x2/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable densenet121/dense_block4/conv_block16/x2/BatchNorm/gamma:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block4/conv_block16/x2/BatchNorm/gamma
INF

INFO:tensorflow:Initialize variable densenet121/dense_block3/conv_block7/x1/BatchNorm/gamma:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block3/conv_block7/x1/BatchNorm/gamma
INFO:tensorflow:Initialize variable densenet121/dense_block4/conv_block11/x1/BatchNorm/gamma:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block4/conv_block11/x1/BatchNorm/gamma
INFO:tensorflow:Initialize variable densenet121/dense_block3/conv_block15/x1/BatchNorm/moving_variance:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block3/conv_block15/x1/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable densenet121/dense_block2/conv_block4/x2/Conv/weights:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block2/conv_block4/x2/Conv/weights
INFO:tensorflow:Initialize variable densenet121/dense_block4/conv_block2/x2/BatchNorm/beta:0 from checkpoint tf-densenet121.ckpt with densenet121/dense_block4/conv_block2/x2/BatchNorm/beta
INFO:tensorflow:Initialize va

In [20]:
%%time
# Main prediction loop: 1min43s
# Test AUC: 0.8037
predictions = list(nn.predict(test_input_fn))
y_truth = test_dataset.labels
y_guess = np.array(predictions)
print("Test AUC: {0:.4f}".format(compute_roc_auc(y_truth, y_guess, CLASSES))) 

INFO:tensorflow:Restoring parameters from /tmp/tmp6esvh3lq/model.ckpt-3410
Full AUC [0.8086564186531395, 0.8452437734054568, 0.7963238015662735, 0.883270318587128, 0.8710611139849179, 0.908246227929895, 0.7131790957365657, 0.8568308874912649, 0.6215392707863726, 0.8299632377662584, 0.7379957407063066, 0.7841262696358536, 0.7239505547425169, 0.8707772921804223]
Test AUC: 0.8037
CPU times: user 17min 29s, sys: 21.9 s, total: 17min 51s
Wall time: 1min 43s
